In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv
import sys
load_dotenv()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path)

['/opt/anaconda3/envs/project-nova/lib/python311.zip', '/opt/anaconda3/envs/project-nova/lib/python3.11', '/opt/anaconda3/envs/project-nova/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/project-nova/lib/python3.11/site-packages', '/Users/vamsisaigarapati/Documents/github/project-nova']


In [2]:
# Standard library imports
import os
import sys
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
print(sys.path)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path)

# Local application imports
from src.config import HEARST_DIR, HEARST_RAW_DIR, HEARST_PROCESSED, HEASRT_FILE, HEASRT_FILE_SISENSE,MSP_AGENNT_LOOKUP_FILE,HEARST_LOOKUP_DIR,MSP_NOT_ASSIGNED_FILE_NAME
from src.configs.hearst_configs import raw_column_types,sisense_columns
from src.utils.excel_file_operations import load_excel_file, write_df_to_excel
from src.utils.dataframe_utils import rearrange_columns

['/opt/anaconda3/envs/project-nova/lib/python311.zip', '/opt/anaconda3/envs/project-nova/lib/python3.11', '/opt/anaconda3/envs/project-nova/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/project-nova/lib/python3.11/site-packages', '/Users/vamsisaigarapati/Documents/github/project-nova']
['/opt/anaconda3/envs/project-nova/lib/python311.zip', '/opt/anaconda3/envs/project-nova/lib/python3.11', '/opt/anaconda3/envs/project-nova/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/project-nova/lib/python3.11/site-packages', '/Users/vamsisaigarapati/Documents/github/project-nova', '/Users/vamsisaigarapati/Documents/github/project-nova']


In [3]:
def calculate_revenue(raw_df):
    """
    Reads the 'Hearst Pub Market List' sheet from 'Heast Files.xlsx',
    joins with raw_df on 'Pub', creates 'Job Number +',
    and returns a DataFrame grouped by 'Job Number +' with summed Revenue.
    Output columns: ['Old Job Number', 'Job Number', 'Job Number +', 'Revenue'].
    """

    market_list = load_excel_file(
        path=HEARST_RAW_DIR,               
        file_name=HEASRT_FILE,
        sheet_name="Hearst Pub Market List",                
    )
    
    merged_df = raw_df.copy()
    market_list = market_list.copy()
    merged_df["Pub_key"] = merged_df["Pub"].astype(str).str.strip().str.lower()
    market_list["Pub_key"] = market_list["Pub"].astype(str).str.strip().str.lower()

    # Merge Market info (inner join for exact matches)
    merged_df = merged_df.merge(
        market_list[["Pub_key", "Market"]],
        on="Pub_key",
        how="inner"
    )

    # Normalize columns for concatenation
    merged_df["Job Number"] = merged_df["Job Number"].astype(str).str.strip()
    merged_df["Market"] = merged_df["Market"].astype(str).str.strip()

    # Create "Job Number +" = Market + Job Number
    merged_df["Job Number +"] = merged_df.apply(
        lambda r: f"{r['Market']}{r['Job Number']}"
        if r["Market"] not in ["", "nan", "None"]
        else r["Job Number"],
        axis=1
    )

    # Convert Revenue safely to numeric
    merged_df["Sum of 'Revenue'"] = pd.to_numeric(merged_df["Revenue"], errors="coerce").fillna(0)

    # ---- Aggregate ----
    # Keep first record for every column except the revenue column
    agg_dict = {col: "first" for col in merged_df.columns if col not in ["Sum of 'Revenue'"]}
    agg_dict["Sum of 'Revenue'"] = "sum"

    result_df = merged_df.groupby("Job Number +", as_index=False).agg(agg_dict)

    # Add the count of records per group
    counts = merged_df.groupby("Job Number +").size().reset_index(name="Count of matches")
    result_df = result_df.merge(counts, on="Job Number +", how="left")
    print(result_df.columns)

    # Reorder columns — keep Job Number +, Sum of Revenue, Count of matches at the end
    cols = [c for c in result_df.columns if c not in ["Job Number +", "Sum of 'Revenue'", "Count of matches"]]
    result_df = result_df[cols + ["Job Number +", "Sum of 'Revenue'", "Count of matches"]]
    result_df['Job Number +'], result_df['Job Number'] = result_df['Job Number'].copy(), result_df['Job Number +'].copy()

    result_df = result_df[result_df["Sum of 'Revenue'"] != 0.0]
    return result_df


def tag_msp_from_rep(processed_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds an 'MSP/non-MSP' flag to processed_df by joining with rep_list.

    Logic:
    1. Load rep_list (MSP agent lookup sheet).
    2. Create lowercase helper columns for matching.
    3. Filter rep_list where 'System(s)' contains 'hearst'
       and exclude agent 'wave2, wave2'.
    4. Deduplicate by Agent Names (first occurrence only).
    5. Left join processed_df.Full Name LF with rep_list.Agent Names (lowercase helper columns).
    6. Add 'MSP/non-MSP' = 'MSP' if match found, else 'Non-MSP'.
    7. Drop helper columns to return original unmodified columns.

    Returns:
        pd.DataFrame: processed_df with a new column 'MSP/non-MSP'
    """

    # --- Step 1: Load the rep_list from Excel lookup ---
    rep_list = load_excel_file(
        path=HEARST_LOOKUP_DIR,
        file_name=MSP_AGENNT_LOOKUP_FILE,
        sheet_name="All Rep Names",
    )

    # --- Step 2: Create lowercase helper columns (preserve originals) ---
    rep_list["_system_lower"] = rep_list["System(s)"].astype(str).str.lower().str.strip()
    rep_list["_agent_lower"] = rep_list["Agent Names"].astype(str).str.lower().str.strip()
    rep_list["_fullname_lower"] = rep_list["Full Name"].astype(str).str.lower().str.strip()

    processed_df["_fullname_lf_lower"] = processed_df["Full Name LF"].astype(str).str.lower().str.strip()

    # --- Step 3: Filter rep_list for 'hearst' but exclude agent 'wave2, wave2' ---
    rep_filtered = rep_list[
        rep_list["_system_lower"].str.contains("hearst", na=False)
        & (rep_list["_agent_lower"] != "wave2, wave2")
    ].copy()

    # --- Step 4: Keep only relevant columns & deduplicate by Agent Name ---
    rep_filtered = (
        rep_filtered[["Agent Names", "System(s)", "Full Name", "_agent_lower"]]
        .drop_duplicates(subset=["_agent_lower"], keep="first")
        .reset_index(drop=True)
    )

    print(f"Filtered rep_list to {len(rep_filtered)} records for 'hearst' system (excluding 'wave2, wave2').")

    # --- Step 5: Perform left join on lowercase helper columns ---
    merged = processed_df.merge(
        rep_filtered[["_agent_lower"]].rename(columns={"_agent_lower": "_join_key"}),
        how="left",
        left_on="_fullname_lf_lower",
        right_on="_join_key",
        indicator=True
    )

    # print(f"Merged DataFrame has {len(merged)} records after join.")

    # --- Step 6: Add MSP flag ---
    merged["MSP/non-MSP"] = merged["_merge"].map({
        "both": "MSP",
        "left_only": "Non-MSP"
    })

    # --- Step 7: Cleanup helper columns ---
    merged = merged.drop(columns=["_fullname_lf_lower", "_join_key", "_merge"], errors="ignore")

    return merged

In [4]:
partner_name = "Hearst"
print(f"Processing data for partner: {partner_name}")
raw_df = load_excel_file(
    path=HEARST_RAW_DIR,                 # or "/full/path/to/dir"
    file_name=HEASRT_FILE,
    column_types=raw_column_types,
    sheet_name="Raw",                    # or omit to read the first sheet
)
# write_df_to_excel(raw_df, HEARST_PROCESSED, "checking.xlsx", sheet_name="Sisense")
processed_df=calculate_revenue(raw_df)
processed_df= tag_msp_from_rep(processed_df)

Processing data for partner: Hearst
Index(['Year', 'Period #', 'Job Number', 'Child Acct #', 'Inches', 'Ad Type',
       'Section', 'Class Code', 'WoRev Bill Cycle', 'Child Acct Name',
       'First Issue Date', 'Full Name LF', 'Business Unit GL', 'GL_LOB_L1',
       'Pub', 'Revenue', 'Pub_key', 'Market', 'Job Number +',
       'Sum of 'Revenue'', 'Count of matches'],
      dtype='object')
Filtered rep_list to 175 records for 'hearst' system (excluding 'wave2, wave2').


In [5]:
from pathlib import Path
from typing import List, Union

import pandas as pd

def enrich_with_msp_reference(
    processed_df: pd.DataFrame,
    *,
    lookup_path: Union[str, Path],
    lookup_file_name: str,
    lookup_sheet_name: str,
) -> pd.DataFrame:
    """
    Update MSP-related fields using a reference Excel file.

    Steps
    -----
    1. Override Wave2 rows so they are always Non-MSP with a fixed name.
    2. Load the MSP lookup and left-join on job number.
    3. For matched rows, replace Full Name LF with the lookup MSP Agent and flag as Non-MSP.

    Parameters
    ----------
    processed_df : pd.DataFrame
        DataFrame produced by the pipeline that needs MSP enrichment.
    lookup_path : str | Path
        Directory where the lookup Excel file lives.
    lookup_file_name : str
        Name of the lookup Excel file (e.g., "MappingFile-Agent Name.xlsx").
    lookup_sheet_name : str
        Sheet inside the lookup Excel to read (e.g., "All Rep Names").

    Returns
    -------
    pd.DataFrame
        A copy of the processed_df with MSP fields updated.
    """
    result_df = processed_df.copy()

    expected_processed_cols = {"Section", "Full Name LF", "MSP/non-MSP", "Job Number +"}
    missing_processed = expected_processed_cols - set(result_df.columns)
    if missing_processed:
        missing = ", ".join(sorted(missing_processed))
        raise KeyError(f"Processed DataFrame missing expected columns: {missing}")

    lookup_df = load_excel_file(
        path=lookup_path,
        file_name=lookup_file_name,
        sheet_name=lookup_sheet_name,
    )

    expected_lookup_cols = {"Job #", "MSP Agent"}
    missing_lookup = expected_lookup_cols - set(lookup_df.columns)
    if missing_lookup:
        missing = ", ".join(sorted(missing_lookup))
        raise KeyError(f"Lookup DataFrame missing expected columns: {missing}")

    wave2_mask = result_df["Section"].fillna("").astype(str).str.contains("Wave2 Death Notices", na=False)
    result_df.loc[wave2_mask, "Full Name LF"] = "Wave2, Wave2"
    result_df.loc[wave2_mask, "MSP/non-MSP"] = "Non-MSP"

    result_df["_join_index"] = result_df.index
    merged = result_df.merge(
        lookup_df[["Job #", "MSP Agent"]],
        how="left",
        left_on="Job Number +",
        right_on="Job #",
    )

    joined_mask = merged["MSP Agent"].notna()
    if joined_mask.any():
        join_indices = merged.loc[joined_mask, "_join_index"]
        new_names = merged.loc[joined_mask, "MSP Agent"].values
        result_df.loc[join_indices, "Full Name LF"] = new_names
        result_df.loc[join_indices, "MSP/non-MSP"] = "Non-MSP"

    result_df.drop(columns="_join_index", inplace=True)

    return result_df


In [6]:
enrich_with_msp_reference(
    processed_df=processed_df,
    lookup_path=HEARST_LOOKUP_DIR,
    lookup_file_name=MSP_NOT_ASSIGNED_FILE_NAME,
    lookup_sheet_name="Not Assigned Reference List",
)

,Year,Period #,Job Number,Child Acct #,Inches,Ad Type,Section,Class Code,WoRev Bill Cycle,Child Acct Name,...,Business Unit GL,GL_LOB_L1,Pub,Revenue,Pub_key,Market,Job Number +,Sum of 'Revenue',Count of matches,MSP/non-MSP
0,2025,8,FF2888723,156557,3.44,CLS Liner,Rentals,05600,Classified Commercial,"WESTPORT, CT BUILDING LOTS",...,3004,Newspapers Traditional,Fairfield Citizen Ne,15.40,fairfield citizen ne,FF,2888723,30.80,2,MSP
1,2025,8,FF2892888,142168,1.56,Legal Liners,Public Notices,11030,Classified Commercial,JP MORGAN CHASE BANK,...,3006,Newspapers Digital,newstimes.com,10.00,newstimes.com,FF,2892888,105.20,2,MSP
2,2025,8,FF2917124,373610,0.01,Online Only,LocalEdge Social Med,OL Retail,Classified Commercial,TOWN OF WALLINGFORD,...,3004,Newspapers Digital,Connpost.com,500.00,connpost.com,FF,2917124,500.00,1,MSP
3,2025,8,FF2918533,146439,0.50,CLS Liner,Service Directory,07346,Retail,TC-PCS,...,3005,Newspapers Digital,stamfordadvocate.com,0.00,stamfordadvocate.com,FF,2918533,20.00,6,Non-MSP
4,2025,8,FF2920793,129375,0.00,Legal Display,Public Notices,11030,Classified Commercial,DANBURY LEGISLATIVE ASSISTANT,...,3006,Newspapers Traditional,Danbury News-Times,-1032.20,danbury news-times,FF,2920793,-1032.20,1,Non-MSP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,2025,8,WA2944179,366534,10.80,CLS Display,Wave2 Death Notices,Life Tributes,Classified Commercial,CASEY FUNERAL HOME,...,3004,Newspapers Digital,Rep-Am.com,35.62,rep-am.com,WA,2944179,280.14,3,Non-MSP
2440,2025,8,WA2944180,125602,13.20,CLS Display,Wave2 Death Notices,Life Tributes,Classified Commercial,WOODTICK MEMORIAL,...,3004,Newspapers Digital,Rep-Am.com,48.79,rep-am.com,WA,2944180,370.81,3,Non-MSP
2441,2025,8,WA2944181,326509,16.80,CLS Display,Wave2 Death Notices,Life Tributes,Classified Commercial,PANAGIOTA,...,3004,Newspapers Digital,Rep-Am.com,67.24,rep-am.com,WA,2944181,497.76,3,Non-MSP
2442,2025,8,WA2944189,370930,14.40,CLS Display,Wave2 Death Notices,Life Tributes,Classified Commercial,STEPHANIE,...,3004,Newspapers Digital,Rep-Am.com,54.06,rep-am.com,WA,2944189,407.08,3,Non-MSP
